#Using SKLL to Test Models/Generate Predictions

In [1]:
import skll

In [2]:
# Let's find out where we are, first, and then see what models we have
from os import getcwd, listdir
getcwd()

'/home/mulhollandm2/reviews_project/reviewer_experience_prediction/ipython_notebooks'

In [3]:
from os.path import dirname, join
listdir(dirname(getcwd()))

['working',
 'results',
 'README',
 'models',
 '.gitignore',
 'arff_files_original_values',
 'data',
 'reports',
 'config',
 'logs',
 'predictions',
 '.git',
 '.ipynb_checkpoints',
 'ipython_notebooks',
 'test',
 'reviews_env',
 'src',
 'util',
 'arff_files_collapsed_values',
 'Makefile']

In [4]:
# There is a "models" directory that contains model files that we can
# try to open with SKLL and use here
# Now let's see what models we have (because models consist of many
# files, let's just get the one with ".model" as a suffix)
from os.path import join
[print(d) for d in listdir(join(dirname(getcwd()),
                                'models')) if d.endswith('.model')]

Arma_3_first500.train_Arma_3_first500_RescaledSVR.model
Football_Manager_2015.train_Football_Manager_2015_RescaledSVR.model
Football_Manager_2015.RSVRquadr.train_Football_Manager_2015_RescaledSVR.model


[None, None, None]

In [5]:
# Alright, so we'll use the Football_Manager_2015 model
l = skll.Learner.from_file(join(dirname(getcwd()),
                                'models',
                                'Football_Manager_2015.train_Football_Manager_2015_RescaledSVR.model'))

In [6]:
# According to the documentation for skll.Learner objects, they have
# a 'predict' method that requires an argument that is an object of
# type FeatureSet, so we'll try to do that next
l.predict?

In [7]:
# We are going to need to get some example data at some point,
# so let's just do it right now
import pymongo
connection_string = 'mongodb://localhost:27017'
connection = pymongo.MongoClient(connection_string)
db = connection['reviews_project']
reviewdb = db['reviews']
reviewdb.write_concern['w'] = 0

In [8]:
r1 = reviewdb.find_one({'game': 'Football_Manager_2015',
                        'partition': 'test'})
print(r1)

{'game': 'Football_Manager_2015', 'hours': 440.0, 'review': 'LOVING THE BETA CANT WAIT FOR FULL GAME ON THE 7TH', 'partition': 'test', 'hours_bin': 5, 'appid': '295270', '_id': ObjectId('5554f294c134cf3ebe2f5009')}


In [9]:
# So we have an example test review and now we can work on
# getting it into an object that can be passed to the prediction
# function
# But, first we will need to calculate all the features
import sys
sys.path.append('/home/mulhollandm2/reviews_project/reviewer_experience_prediction/')
from src.feature_extraction import Review, extract_features_from_review

In [11]:
# Prints out the documentation for this class that is part of
# our code
r1_review_obj = Review?

In [12]:
r1_review_obj = Review

In [13]:
from spacy.en import English
spaCy_nlp = English()
r1_review_obj = Review(r1['review'],
                       float(r1['hours_bin']), # using bins, i.e.,
                                               # collapsed, form of
                                               # the hours played
                                               # value
                       r1['game'],
                       r1['appid'],
                       spaCy_nlp,
                       )

In [14]:
# Let's take a look at the object
print(r1_review_obj.length)

6


In [15]:
# Here's the full list of attributes that are contained in each
# Review object (attributes being variables or methods)
[a for a in dir(r1_review_obj) if not a.startswith('__')]

['appid',
 'get_entities_from_spaCy',
 'get_token_features_from_spaCy',
 'hours_played',
 'length',
 'lower',
 'norm',
 'normalize',
 'orig',
 'spaCy_annotations',
 'spaCy_sents',
 'tags',
 'tokens']

In [16]:
print(r1_review_obj.tokens)

[['loving', 'the', 'beta', 'can', 'not', 'wait', 'for', 'full', 'game', 'on', 'the', '7th']]


In [17]:
# With the processed representations of the review text, now we
# can proceed to extract features
# Let's see the documentation first
r1_features = extract_features_from_review?

In [18]:
r1_features = extract_features_from_review

In [19]:
r1_features = extract_features_from_review(r1_review_obj)
r1_features

{' 7': 1,
 ' 7T': 1,
 ' 7TH': 1,
 ' B': 1,
 ' BE': 1,
 ' BET': 1,
 ' BETA': 1,
 ' C': 1,
 ' CA': 1,
 ' CAN': 1,
 ' CANT': 1,
 ' F': 2,
 ' FO': 1,
 ' FOR': 1,
 ' FOR ': 1,
 ' FU': 1,
 ' FUL': 1,
 ' FULL': 1,
 ' G': 1,
 ' GA': 1,
 ' GAM': 1,
 ' GAME': 1,
 ' O': 1,
 ' ON': 1,
 ' ON ': 1,
 ' ON T': 1,
 ' T': 2,
 ' TH': 2,
 ' THE': 2,
 ' THE ': 2,
 ' W': 1,
 ' WA': 1,
 ' WAI': 1,
 ' WAIT': 1,
 '6': 1,
 '7T': 1,
 '7TH': 1,
 '7th': 1,
 '7th:the': 1,
 'A ': 1,
 'A C': 1,
 'A CA': 1,
 'A CAN': 1,
 'AI': 1,
 'AIT': 1,
 'AIT ': 1,
 'AIT F': 1,
 'AM': 1,
 'AME': 1,
 'AME ': 1,
 'AME O': 1,
 'AN': 1,
 'ANT': 1,
 'ANT ': 1,
 'ANT W': 1,
 'BE': 1,
 'BET': 1,
 'BETA': 1,
 'BETA ': 1,
 'CA': 1,
 'CAN': 1,
 'CANT': 1,
 'CANT ': 1,
 'E ': 3,
 'E 7': 1,
 'E 7T': 1,
 'E 7TH': 1,
 'E B': 1,
 'E BE': 1,
 'E BET': 1,
 'E O': 1,
 'E ON': 1,
 'E ON ': 1,
 'ET': 1,
 'ETA': 1,
 'ETA ': 1,
 'ETA C': 1,
 'FO': 1,
 'FOR': 1,
 'FOR ': 1,
 'FOR F': 1,
 'FU': 1,
 'FUL': 1,
 'FULL': 1,
 'FULL ': 1,
 'G ': 1,
 'G T': 1,


In [20]:
print(len(r1_features))

210


In [21]:
# As you can see, even for a short review, there are 210
# features present
# Now, how do we put this in an object that we can pass to the
# prediction function?

In [23]:
# For one thing, I just want to re-check the version of SKLL
# being used
print(skll.version.VERSION)

(1, 0, 1)


In [24]:
# Alright, great, we are using version 1.0.1, as I thought

In [25]:
from skll.data.featureset import FeatureSet
import numpy as np

In [39]:
r1_fs = FeatureSet('single_review_Football_Manager_2015',
                   np.array([str(r1['_id'])],
                            dtype=np.chararray),
                   np.array([float(r1['hours_bin'])],
                            dtype=np.float32),
                   [r1_features])

In [40]:
# Let's see what r1_fs is
r1_fs.features

<1x210 sparse matrix of type '<class 'numpy.float64'>'
	with 210 stored elements in Compressed Sparse Row format>

In [42]:
r1_fs.ids

array(['5554f294c134cf3ebe2f5009'], dtype=object)

In [43]:
r1_fs.name

'single_review_Football_Manager_2015'

In [45]:
# Ok, let's try to predict the label with our learner
l.predict(r1_fs)

array([ 1.94951896])

In [46]:
# How does the prediction compare with the actual value? Let's
# see...
r1_review_obj.hours_played

5.0

In [47]:
# So, it is off by a few. Remember, this is a 10-point scale, so
# the difference isn't quite as drastic as it might initially appear,
# but it is quite different